In [ ]:
from IPython.core.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
import datetime
import eqcatalog

In [ ]:
full_cat = eqcatalog.rob.query_local_eq_catalog(region=(0, 8, 49, 52), event_type='all')
## Add converted moment magnitude column
moment_mags = full_cat.get_magnitudes('MW', Mrelation={})
for i in range(len(full_cat)):
    full_cat[i].set_mag('MWc', moment_mags[i])
full_cat.default_Mrelations = {}

In [ ]:
date_slider = widgets.IntRangeSlider(
    value=[1910, 2020],
    min=1350,
    max=2020,
    step=1,
    description='Year range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
#display(date_slider)

In [ ]:
start_date_picker = widgets.DatePicker(
    value=datetime.date(1985, 1, 1),
    description='Start date:',
    disabled=False
)
#display(start_date_picker)

In [ ]:
end_date_picker = widgets.DatePicker(
    value = datetime.date.today(),
    description='End date:',
    disabled=False
)
#display(end_date_picker)

In [ ]:
mtype_selector = widgets.Dropdown(
    options=['ML', 'MS', 'MW', 'MWc'],
    value='ML',
    description='Mag. type:',
    disabled=False,
)
#display(mtype_selector)

In [ ]:
lon_slider = widgets.FloatRangeSlider(
    value=[0., 8.],
    min=-1.,
    max=9.,
    step=0.1,
    description='Lon range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [ ]:
lat_slider = widgets.FloatRangeSlider(
    value=[49., 52.],
    min=48.,
    max=53.,
    step=0.1,
    description='Lat range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [ ]:
mag_slider = widgets.FloatRangeSlider(
    value=[2., 7.0],
    min=-1.,
    max=7.0,
    step=0.1,
    description='Magnitude:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
#display(mag_slider)

In [ ]:
depth_slider = widgets.IntRangeSlider(
    value=[0, 50],
    min=0,
    max=50,
    step=1,
    description='Depth range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
#display(depth_slider)

In [ ]:
etype_selector = widgets.SelectMultiple(
    options=[('Earthquake', 'ke'),
            ('Suspected earthquake', 'se'),
            ('Induced event', 'ki'),
            ('Suspected induced event', 'si'),
            ('Quarry blast', 'qb'),
            ('Suspected quarry blast', 'sqb'),
            ('Controlled explosion', 'cb'),
            ('Suspected controlled explosion', 'scb'),
            ('Mine explosion', 'km'),
            ('Suspected mine explosion', 'sm')],
    value=['ke'],
    #rows=10,
    description='Event type:',
    disabled=False
)
#display(etype_selector)

In [ ]:
plot_type_selector = widgets.Dropdown(
    options=['basemap', 'folium'],
    value='folium',
    description='Plot type:',
    disabled=False,
)
#display(plot_type_selector)

In [ ]:
folium_map = None

In [ ]:
## Seismotectonic source model
folium_layers = []
try:
    import hazard.rshalib as rshalib
except:
    pass
else:
    somo = rshalib.rob.read_source_model('Seismotectonic_Hybrid')
    az_model = rshalib.source.SourceModel('Seismotectonic zones', somo.get_area_sources())
    flt_model = rshalib.source.SourceModel('RVRS faults', somo.get_fault_sources())
    for somo in [az_model, flt_model]:
        folium_layers.append(somo.to_folium_layer(area_fill_color='green', area_stroke_color='green',
                                                  area_fill_opacity=0.2,
                                                  fault_geom_type='both', show_popups=False))


In [ ]:
def subselect_catalog(plot_type, min_date, max_date, lon_range, lat_range,
                      mag_range, Mtype, depth_range, event_types):
    global folium_map
    
    if plot_type == 'folium' and folium_map:
        ## Unfortunately, folium map bounds are not updated, so this doesn't work...
        ((lat_min, lon_min), (lat_max, lon_max)) = folium_map.get_bounds()
        lon_range = [lon_min, lon_max]
        lat_range = [lat_min, lat_max]

    region = lon_range + lat_range
    min_mag, max_mag = mag_range
    if min_mag == mag_slider.min and max_mag == mag_slider.max:
        ## Include undetermined magnitudes
        min_mag = max_mag = None
    min_depth, max_depth = depth_range
        ## Include undetermined depths
    if min_depth == depth_slider.min and max_depth == depth_slider.max:
        min_depth = max_depth = None

    subcat = full_cat.subselect(start_date=min_date, end_date=max_date,
                               region=region,
                               Mmin=min_mag, Mmax=max_mag, Mtype=Mtype,
                               min_depth=min_depth, max_depth=max_depth,
                               attr_val=('event_type', list(event_types)))
    subcat.name = 'ROB Catalog'
    if plot_type == 'basemap':
        ax = subcat.plot_map(region=region, Mtype=Mtype, Mrelation={})
    elif plot_type == 'folium':
        folium_map = subcat.get_folium_map(Mtype=Mtype, Mrelation={}, region=region,
                                          additional_layers=folium_layers)
        display(folium_map)

In [ ]:
interactive_plot = interactive(subselect_catalog,
                               plot_type=plot_type_selector,
                               #year_range=date_slider,
                               min_date=start_date_picker, max_date=end_date_picker,
                               lon_range=lon_slider, lat_range=lat_slider,
                               mag_range=mag_slider, Mtype=mtype_selector,
                               depth_range=depth_slider,
                               event_types=etype_selector)
#output = interactive_plot.children[-1]
#output.layout.height = '350px'
interactive_plot